# ML Pipeline testing
---
---

# Imports

#### Standard library imports

In [1]:
import sys
import pickle

sys.path.append("../../")

#### Third party imports

In [2]:
import pandas as pd

#### Local application imports

In [4]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extraction

In [ ]:
extract_pipeline_func()

## Results

In [ ]:
pkl_obj = pipeline_pkl_extract_path + "/" + pipeline_pkl_extract_name

In [ ]:
with open(pkl_obj, 'rb') as obj_content:
    dfx = pickle.load(obj_content)

# Transformation

In [6]:
transform_pipeline_func()

## Results

In [7]:
pkl_obj = pipeline_pkl_extract_path + "/" + pipeline_pkl_extract_name

In [8]:
with open(pkl_obj, 'rb') as obj_content:
    dfx = pickle.load(obj_content)

In [9]:
dfx

,appointment_id,appointment_date
0,2023001189807,2023-12-15
1,2023001189857,2023-12-27
2,2023001189846,2023-12-22
3,2023001189931,2023-12-19
4,2023001189932,2023-12-19
...,...,...
6958,2023001451480,2023-12-20
6959,2023001326688,2023-12-26
6960,2023001326686,2023-12-26
6961,2023001326690,2023-12-26


# Notes

## Generate data schema

In [ ]:
df_cols = dfx.columns.tolist()
data_dicts_loc = 'pkg_dir/src/parameters/'
data_dict_filename = 'data_schema.txt'

In [ ]:
generate_data_dictionary(df_cols, data_dicts_loc, data_dict_filename)

---

---